# Predicting the Reviews of a Movie with IMDB
# Patrick Wang



## Introduction
IMDB is a website where users and critics alike can review movies and input their opinions where everyone can see. While it supports more features than just reviews, IMDB is predominantly used in conjunction with other movie review websites like Rotten Tomatos to predict if a movie will be good or bad before people even watch them. Users can leave a positive or negative review and attach a short phrase that summarizes their viewpoints. Many times, a positive or negative aggregate score on these movie review websites can heavily influence people into watching the movie or not. For those on the fence about watching a movie, a positive review by another user could make or break their decision.
In this tutorial, we will be attempting to use a dataset from IMDB with positive and negative reviews and try to extract insight as to if a movie review will be good or bad.

In this tutorial, we will be attempting to use a dataset from IMDB with positive and negative reviews and try to extract insight as to if a movie review will be good or bad. 

First, we must download the IMDB reviews dataset and read it with pandas.
Dataset retrieved from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
import pandas as pd
movie_set = pd.read_csv('IMDB Dataset.csv')
movie_set

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


Since there are 50000 rows, in order to speed up the process, we will instead use 2000 rows, 1000 positives and 1000 negatives.

In [2]:
negatives = movie_set[movie_set['sentiment'] == 'negative'][:1000]
positives = movie_set[movie_set['sentiment'] == 'positive'][:1000]
movie_set_total = pd.concat([positives, negatives])

Next, we need to create a training set and a testing set. The training set's purpose is to help the model learn and the testing set is to see how well the model works after it is trained. We will use 33% of the data to test and can set the variables within the datasets. train_x is what we use to train the model, and since we dedicated 33% to testing, this one will have 67% of observations. train_y is what we are trying to predict, test_x is the 33% of variables we will use to test the model, and test_y is what we will use to test the accuracy of our model.

In [3]:
from sklearn.model_selection import train_test_split
training_set, testing_set = train_test_split(movie_set_total, test_size = 0.33, random_state = 42)
training_set_x, training_set_y = training_set['review'], training_set['sentiment']
testing_set_x, testing_set_y = testing_set['review'], testing_set['sentiment']

Since the learning algorithms require numbers instead of text, we have to find a way to turn our reviews into numericals. We can use Bag of Words to convert our review texts into numbers. We will be representing the texts as what is called Term Frequency, Inverse Document Frequency. This method gives weights to words in a collection depending on how important it is. The importance increases as the frequency of the word increases. We will use TfidVectorizer to remove english stopwords and use pandas to display the matrix. Also, add an extra line of code at the bottom for our future testing purposes.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
post_tfid = TfidfVectorizer(stop_words = 'english')
training_x_vector = post_tfid.fit_transform(training_set_x)
training_x_vector

pd.DataFrame.sparse.from_spmatrix(training_x_vector, index = training_set_x.index, columns = post_tfid.get_feature_names())
testing_x_vector = post_tfid.transform(testing_set_x)

We are done with the data collection part, so now we can focus on finding an appropriate machine learning model. We will decide between Decision Tree and Support Vector Machines. We first try predicting the movie review with our own tests, and can see that SVM returns the correct predictions for our small tests. We can also use the score method to get a number representing their accuracy. It is then evident that SVM is more accurate (.837 > .678).

In [5]:
from sklearn.tree import DecisionTreeClassifier
dTree = DecisionTreeClassifier()
dTree.fit(training_x_vector, training_set_y)
print(dTree.predict(post_tfid.transform(['This was a good movie'])))
print(dTree.predict(post_tfid.transform(['It was alright'])))
print(dTree.predict(post_tfid.transform(['A bad movie'])))
dTree.score(testing_x_vector, testing_set_y)

['positive']
['positive']
['negative']


0.6787878787878788

In [6]:
from sklearn.svm import SVC
SVM = SVC(kernel='linear')
SVM.fit(training_x_vector, training_set_y)
print(SVM.predict(post_tfid.transform(['This was a good movie'])))
print(SVM.predict(post_tfid.transform(['It was alright'])))
print(SVM.predict(post_tfid.transform(['A bad movie'])))
SVM.score(testing_x_vector, testing_set_y)

['positive']
['positive']
['negative']


0.8378787878787879

We can go further and calculate the F1 score, which is the weighted average of how many true positives were found and how many of the returned results were true positive. These are commonly called Recall and Precision, respectively. Since false negatives and false positives are expected in our set, so finding the F1 score could help us in further measuring our method accuracies. The F1 score for SVM is calculated to be .84 while the Decision Tree's is .83. With this we can conclude that the SVM method is more accurate and precise.

In [7]:
from sklearn.metrics import f1_score
f1_score(testing_set_y, SVM.predict(testing_x_vector), labels = ['positive', 'negative'], average = None)

array([0.8410104 , 0.83462133])

Additionally, we can use a confusion matrix to visualize the accuracy of our algorithms. The returned array is made up of the number of false positives, false negatives, true positives, and true negatives. The top left number (283) is the false positives, the top right number (45) is the false negatives, the bottom left number (65) is the true positives, and the bottom right number (270) is the true negatives. 

In [8]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(testing_set_y, SVM.predict(testing_x_vector), labels = ['positive', 'negative'])
conf_matrix

array([[283,  42],
       [ 65, 270]])

## Conclusion and Further Exploration

IMDB is one of the biggest movie review websites in existence, and is one of the first sources of info consumers use when deciding to watch a movie or not. Even if the consumer has no doubts they will watch the movie, even if they search the movies name the IMDB score will often pop up without even being asked. As a result, the IMDB scores are heavily influential in how a movie is perceived by the public. These scores can influence public opinion, and are actually one of the ways producers and studios can see how their movie is received, aside from box office statistics. Similar methods of what we have just done can be applied to other review websites aside from IMDB, such as other movie review websites, retail product reviews, and basically anything with a user review system. 

In the future, the methods can be expanded to account for ambiguous reviews, such as one that only contains emojis, or reviews that heavily use sarcasm. Similar methods can be adapted to websites that ask for user reviews, such as big retail companies like Amazon. When we are unsure about a product or service, we oftentimes find ourselves researching and reviewing other customer reviews to make better informed decisions. This project can help consumers better see how well something is reviewed by the public, leading to them making more calculated decisions.